In [30]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import time
import re

urls = [
    'https://emojipedia.org/smileys#list'
    'https://emojipedia.org/people#list',
    'https://emojipedia.org/nature#list',
    'https://emojipedia.org/food-drink#list',
    'https://emojipedia.org/activity#list',
    'https://emojipedia.org/travel-places#list',
    'https://emojipedia.org/objects#list',
    'https://emojipedia.org/symbols#list',
    'https://emojipedia.org/flags#list'
    ]

In [32]:
output = []
cnt = 0

for url in urls:
    response = requests.get(url=url)

    if response and response.ok:
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all('script')
        
        for item in data:
            if 'mutations' in item.text:
                # print(item.text)
                json_data = json.loads(item.text)
                
                # with open('test.json', 'w', encoding='utf-8') as f:
                #     json.dump(json_data, f, ensure_ascii=False, indent=4)
                
                subcats = json_data['props']['pageProps']['dehydratedState']['queries'][3]['state']['data']['subCategories']
                
                for subcat in subcats:
                    print(f" {url} | {subcat['title']}")
                    subcat_len = len(subcat['emoji'])
                                     
                    print(f" Emojis in subcat: {subcat_len}")
                    cnt += subcat_len
                    
                    print(f" Emoji collected: {cnt}\n")
                    
                    for emoji in subcat['emoji']:
                        output.append(emoji)
                
                time.sleep(60)
                    

 https://emojipedia.org/smileys#listhttps://emojipedia.org/people#list | Smiling & Affectionate
 Emojis in subcat: 22
 Emoji collected: 22

 https://emojipedia.org/smileys#listhttps://emojipedia.org/people#list | Tongues, Hands & Accessories
 Emojis in subcat: 19
 Emoji collected: 41

 https://emojipedia.org/smileys#listhttps://emojipedia.org/people#list | Neutral & Skeptical
 Emojis in subcat: 17
 Emoji collected: 58

 https://emojipedia.org/smileys#listhttps://emojipedia.org/people#list | Sleepy & Unwell
 Emojis in subcat: 17
 Emoji collected: 75

 https://emojipedia.org/smileys#listhttps://emojipedia.org/people#list | Concerned & Negative
 Emojis in subcat: 30
 Emoji collected: 105

 https://emojipedia.org/smileys#listhttps://emojipedia.org/people#list | Costume, Creature & Animal
 Emojis in subcat: 24
 Emoji collected: 129

 https://emojipedia.org/nature#list | Mammals & Marsupials
 Emojis in subcat: 66
 Emoji collected: 195

 https://emojipedia.org/nature#list | Birds
 Emojis in s

In [33]:
len(output)

1495

In [34]:
df = pd.DataFrame(output)
df.head()

,id,title,currentCldrName,slug,description,code
0,1,Grinning Face,Grinning Face,grinning-face,"<p>A yellow face with simple, open eyes and a ...",😀
1,4,Smiling Face with Open Mouth,Grinning Face with Big Eyes,grinning-face-with-big-eyes,<p>A yellow face with smiling eyes and a broad...,😃
2,5,Smiling Face with Open Mouth and Smiling Eyes,Grinning Face with Smiling Eyes,grinning-face-with-smiling-eyes,<p>A yellow face with smiling eyes and a broad...,😄
3,2,Grinning Face with Smiling Eyes,Beaming Face with Smiling Eyes,beaming-face-with-smiling-eyes,<p>A yellow face with smiling eyes and full-to...,😁
4,7,Smiling Face with Open Mouth and Tightly-Close...,Grinning Squinting Face,grinning-squinting-face,"<p>A yellow face with a broad, open smile and ...",😆


In [35]:
def clear_html(text,
               verbose:bool = False):
    soup = BeautifulSoup(text)
    clr_text = soup.get_text()
    clr_text = re.sub(r'\s+', ' ', clr_text)
    if verbose:
        print(f"BEFORE: {text}")        
        print(f"AFTER: {repr(clr_text)}")
        
    return clr_text

In [36]:
df['encode-utf-8'] = df['code'].apply(lambda x: x.encode('utf-8'))
df['encode-unicode-escape'] = df['code'].apply(lambda x: x.encode('unicode-escape'))
# df['description'] = df['description'].apply(lambda x: x.replace("<p>", ""))
df['description'] = df['description'].apply(lambda x: clear_html(x))

In [37]:
df.head()

,id,title,currentCldrName,slug,description,code,encode-utf-8,encode-unicode-escape
0,1,Grinning Face,Grinning Face,grinning-face,"A yellow face with simple, open eyes and a bro...",😀,b'\xf0\x9f\x98\x80',b'\\U0001f600'
1,4,Smiling Face with Open Mouth,Grinning Face with Big Eyes,grinning-face-with-big-eyes,"A yellow face with smiling eyes and a broad, o...",😃,b'\xf0\x9f\x98\x83',b'\\U0001f603'
2,5,Smiling Face with Open Mouth and Smiling Eyes,Grinning Face with Smiling Eyes,grinning-face-with-smiling-eyes,"A yellow face with smiling eyes and a broad, o...",😄,b'\xf0\x9f\x98\x84',b'\\U0001f604'
3,2,Grinning Face with Smiling Eyes,Beaming Face with Smiling Eyes,beaming-face-with-smiling-eyes,A yellow face with smiling eyes and full-tooth...,😁,b'\xf0\x9f\x98\x81',b'\\U0001f601'
4,7,Smiling Face with Open Mouth and Tightly-Close...,Grinning Squinting Face,grinning-squinting-face,"A yellow face with a broad, open smile and scr...",😆,b'\xf0\x9f\x98\x86',b'\\U0001f606'


In [38]:
df.drop_duplicates(subset=['id'])

,id,title,currentCldrName,slug,description,code,encode-utf-8,encode-unicode-escape
0,1,Grinning Face,Grinning Face,grinning-face,"A yellow face with simple, open eyes and a bro...",😀,b'\xf0\x9f\x98\x80',b'\\U0001f600'
1,4,Smiling Face with Open Mouth,Grinning Face with Big Eyes,grinning-face-with-big-eyes,"A yellow face with smiling eyes and a broad, o...",😃,b'\xf0\x9f\x98\x83',b'\\U0001f603'
2,5,Smiling Face with Open Mouth and Smiling Eyes,Grinning Face with Smiling Eyes,grinning-face-with-smiling-eyes,"A yellow face with smiling eyes and a broad, o...",😄,b'\xf0\x9f\x98\x84',b'\\U0001f604'
3,2,Grinning Face with Smiling Eyes,Beaming Face with Smiling Eyes,beaming-face-with-smiling-eyes,A yellow face with smiling eyes and full-tooth...,😁,b'\xf0\x9f\x98\x81',b'\\U0001f601'
4,7,Smiling Face with Open Mouth and Tightly-Close...,Grinning Squinting Face,grinning-squinting-face,"A yellow face with a broad, open smile and scr...",😆,b'\xf0\x9f\x98\x86',b'\\U0001f606'
...,...,...,...,...,...,...,...,...
1490,1112,Flag: St. Vincent & Grenadines,Flag: St. Vincent & Grenadines,flag-st-vincent-grenadines,"The flag for the St. Vincent & Grenadines, whi...",🇻🇨,b'\xf0\x9f\x87\xbb\xf0\x9f\x87\xa8',b'\\U0001f1fb\\U0001f1e8'
1491,1144,Flag: Vanuatu,Flag: Vanuatu,flag-vanuatu,"The flag for Vanuatu, which may show as the le...",🇻🇺,b'\xf0\x9f\x87\xbb\xf0\x9f\x87\xba',b'\\U0001f1fb\\U0001f1fa'
1492,1148,Flag: Wallis & Futuna,Flag: Wallis & Futuna,flag-wallis-futuna,"The flag for the Wallis & Futuna, which may sh...",🇼🇫,b'\xf0\x9f\x87\xbc\xf0\x9f\x87\xab',b'\\U0001f1fc\\U0001f1eb'
1493,1086,Flag: Samoa,Flag: Samoa,flag-samoa,"The flag for Samoa, which may show as the lett...",🇼🇸,b'\xf0\x9f\x87\xbc\xf0\x9f\x87\xb8',b'\\U0001f1fc\\U0001f1f8'


In [39]:
jsondata = df.to_json('emojis.jsonl', orient='records', lines=True)
jsondata